Die aktuellen COVID19 Zahlen können unter folgender Seite heruntergeladen werden: https://covid19.ssi.dk/overvagningsdata/download-fil-med-overvaagningdata
<br>
Abgespeichert werden Sie unter data/raw/observations/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/GitHub/master-thesis/data/raw/observations/

/content/drive/MyDrive/GitHub/master-thesis/data/raw/observations


Es werden folgende COVID Zahlen in eine .csv Datei zusammengeführt:


*   number of deaths (.csv)
*   number of tests (.csv)
*   number of hospitalizations (.csv)





In [4]:
df_deaths = pd.read_csv('overvaagningsdata-dashboard-covid19-31012022-40ip/Kommunalt_DB/06_doed_pr_dag_kumuleret.csv', sep=';', thousands='.', decimal=',', encoding='iso-8859-1')
df_deaths.rename(columns={'Dato':'Date', 'Døde':'number_of_deaths', 'Kumuleret antal døde': 'cumulative_number_of_deaths'}, inplace=True)
df_deaths.drop(df_deaths.tail(1).index,inplace=True) # drop last row
df_deaths.head(3)

,Date,number_of_deaths,cumulative_number_of_deaths
0,2020-02-26,0,0
1,2020-02-27,0,0
2,2020-02-28,0,0


In [5]:
df_deaths.tail(1)

,Date,number_of_deaths,cumulative_number_of_deaths
703,2022-01-30,11,3704


In [6]:
df_confirmed_cases = pd.read_csv('overvaagningsdata-dashboard-covid19-31012022-40ip/Regionalt_DB/08_bekraeftede_tilfaelde_pr_dag_pr_regions.csv', sep=';', thousands='.', decimal=',', encoding='iso-8859-1')
df_confirmed_cases.rename(columns={'Dato':'Date', 'Bekræftede tilfælde': 'newly_infected'}, inplace=True)
df_confirmed_cases = df_confirmed_cases.groupby(by='Date').sum().reset_index()
df_confirmed_cases.drop(df_confirmed_cases.tail(1).index,inplace=True) # drop last row
df_confirmed_cases.head(3)

,Date,newly_infected
0,2020-02-26,1
1,2020-02-27,1
2,2020-02-28,1


In [7]:
df_confirmed_cases.tail(1)

,Date,newly_infected
703,2022-01-30,12808


In [8]:
df_hospitalizations = pd.read_csv('overvaagningsdata-dashboard-covid19-31012022-40ip/Kommunalt_DB/05_nye_indlaeggelser_pr_dag.csv', sep=';', thousands='.', decimal=',', encoding='iso-8859-1')
df_hospitalizations.rename(columns={'Indlæggelsesdato': 'Date', 'Indlæggelser': 'hospitalization'}, inplace=True)
df_hospitalizations.head(3)

,Date,hospitalization
0,2020-02-26,0
1,2020-02-27,0
2,2020-02-28,0


In [9]:
df_hospitalizations.tail(1)

,Date,hospitalization
703,2022-01-30,195


Merge Dataframes

In [10]:
df_death_cases = pd.merge(df_deaths, df_confirmed_cases, how='left', on='Date')
df_observations = pd.merge(df_death_cases, df_hospitalizations, how='left', on='Date')
df_observations.head(3)

,Date,number_of_deaths,cumulative_number_of_deaths,newly_infected,hospitalization
0,2020-02-26,0,0,1,0
1,2020-02-27,0,0,1,0
2,2020-02-28,0,0,1,0


In [11]:
df_observations.tail(3)

,Date,number_of_deaths,cumulative_number_of_deaths,newly_infected,hospitalization
701,2022-01-28,18,3677,35946,250
702,2022-01-29,16,3693,28186,218
703,2022-01-30,11,3704,12808,195


In [12]:
df_observations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 704 entries, 0 to 703
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Date                         704 non-null    object
 1   number_of_deaths             704 non-null    int64 
 2   cumulative_number_of_deaths  704 non-null    int64 
 3   newly_infected               704 non-null    int64 
 4   hospitalization              704 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 33.0+ KB


In [13]:
df_observations.to_csv('/content/drive/MyDrive/GitHub/master-thesis/data/220309_observations.csv', encoding='utf-8', sep=';', decimal='.', index=False)

# Calculate R-value

Notizen:
 7 Tage R-Wert für den 11.03.2020 = 
 Summe(06.03-12.03.)/Summe(02.03.-08.03.) <br>
 r(t) = sum(n(t-5):n(t+1)) / sum(n(t-9):n(t-3))
 

In [ ]:
df_observations.head()

In [ ]:
def calc_7day_r_value(row):
  i_day = row.name
  r_value = np.sum(df_observations['Confirmed_cases'].iloc[i_day-5:i_day+2]) / np.sum(df_observations['Confirmed_cases'].iloc[i_day-9:i_day-2])
  return r_value

df_observations['7day_r_value'] = np.nan
df_observations['7day_r_value'][9:] = df_observations[9:].apply(calc_7day_r_value, axis=1)
df_observations.head()